In [2]:
import numpy as np
from numba import njit

In [3]:
# Martin's code to compute Y_lm's
@njit
def slow_recurrence(Nl, Ylm, xx):
    for m in range(0,Nl-1):
        for ell in range(m+2,Nl):
            fact1,fact2  = np.sqrt( (ell-m)*1./(ell+m) ),\
                           np.sqrt( (ell-m-1.)/(ell+m-1.) )
            Ylm[ell,m,:] = (2*ell-1)*xx*Ylm[ell-1,m,:]-\
                           (ell+m-1)   *Ylm[ell-2,m,:]*fact2
            Ylm[ell,m,:]*= fact1/(ell-m)
    return Ylm

def compute_ylm_table(Nell,Nx):
    """Use recurrence relations to compute a table of Ylm[theta,0]
    for ell>=0, m>=0, x=>0."""
    xx = (np.arange(Nx)+0.5)/Nx
    sx = np.sqrt(1-xx**2)
    Ylm= np.zeros( (Nell,Nell,Nx) )
    #
    # First we do the m=0 case.
    Ylm[0,0,:] = np.ones_like(xx)
    Ylm[1,0,:] = xx.copy()
    for ell in range(2,Nl):
        Ylm[ell,0,:] = (2*ell-1)*xx*Ylm[ell-1,0,:]-(ell-1)*Ylm[ell-2,0,:]
        Ylm[ell,0,:]/= float(ell)
    # Now we fill in m>0.
    # To keep the recurrences stable, we treat "high m" and "low m" separately.
    # Start with the highest value of m allowed:
    for m in range(1,Nl):
        Ylm[m,m,:] = -np.sqrt(1.0-1./(2*m))*sx*Ylm[m-1,m-1,:]
    # Now do m=ell-1
    for m in range(1,Nl-1):
        Ylm[m+1,m,:] = np.sqrt(2*m+1.)*xx*Ylm[m,m,:]
    # Finally fill in ell>m+1:
    _ = slow_recurrence(1, Ylm, xx) # A dummy, warmup run to JIT compile
    Ylm = slow_recurrence(Nl, Ylm, xx)
    # And finally put in the (2ell+1)/4pi normalization:
    for ell in range(Nl):
        Ylm[ell,:,:] *= np.sqrt( (2*ell+1)/4./np.pi )
    return(Ylm)

In [4]:
Nl  = 500
Nx  = 1024
theta_samples = (np.arange(Nx)+0.5)/Nx
Ylm_np = compute_ylm_table(Nl,Nx)

In [5]:
# Let's generate some mock data
Nrandoms = int(1e7)
theta_data = np.random.uniform(0,np.pi,Nrandoms)
w_i = np.random.uniform(0,1, Nrandoms)

# We will want to sort the data in ascending order of theta
sorted_indices = np.argsort(theta_data)
theta_data_sorted = theta_data[sorted_indices]
w_i_sorted = w_i[sorted_indices] # The observation weights for each galaxy/random

In [1]:
# Calculate theta_data-theta_sample[i] for each theta_data
which_spline_idx = np.digitize(theta_data_sorted, theta_samples) - 1
digitized_diffs = (theta_data_sorted - theta_samples[which_spline_idx])

NameError: name 'np' is not defined

In [7]:
@njit
def get_sum(x_samples, x_data, y_data):
    sum = np.zeros_like(x_samples)
    j=0
    for x_d, y_d in zip(x_data, y_data):
        if x_d<x_samples[j+1]:
            sum[j] += y_d
        else:
            j+=1
    return sum

# We now sum up all the w_p (theta_data_p-theta_sample[i])^n in each spline region i, for n in {0,1,2,3}, and a galaxy/random labeled by p
summed_0_diffs = get_sum(theta_samples, theta_data_sorted,
                         w_i_sorted * np.ones_like(digitized_diffs))
summed_1_diffs = get_sum(theta_samples, theta_data_sorted,
                         w_i_sorted * digitized_diffs)
summed_2_diffs = get_sum(theta_samples, theta_data_sorted,
                         w_i_sorted * digitized_diffs**2)
summed_3_diffs = get_sum(theta_samples, theta_data_sorted,
                         w_i_sorted * digitized_diffs**3)

print('Note that these take up a negligible amount of memory: {} Mb'.format(4*summed_0_diffs.nbytes/1e6))

Note that these take up a negligible amount of memory: 0.032768 Mb


In [8]:
@njit
def cubic_spline(x, y, ind, t_0, t_1, t_2, t_3, endpoints="natural"):
    """
    :param x: 1d numpy array of x samples
    :param y: 1d numpy array of y samples
    :param ind: 1d numpy array of sample indices -- almost trivial np.arange(len(x))
    :param t_0: 1d numpy array with (data-sample[i])^0 for each spline bin
    :param t_1: 1d numpy array with (data-sample[i])^1 for each spline bin
    :param t_2: 1d numpy array with (data-sample[i])^2 for each spline bin
    :param t_3: 1d numpy array with (data-sample[i])^3 for each spline bin
    :param endpoints: str. "natural" or "not-a-knot"
    :return: 
    """
    
    n_data = len(x)
    # Difference vectors
    h = np.diff(x)  # x[i+1] - x[i] for i=0,...,n-1
    p = np.diff(y)  # y[i+1] - y[i]

    # Special values for the first and last equations
    zero = np.array([0.0])
    one = np.array([1.0])
    A00 = one if endpoints == "natural" else np.array([h[1]])
    A01 = zero if endpoints == "natural" else np.array([-(h[0] + h[1])])
    A02 = zero if endpoints == "natural" else np.array([h[0]])
    ANN = one if endpoints == "natural" else np.array([h[-2]])
    AN1 = (
        -one if endpoints == "natural" else np.array([-(h[-2] + h[-1])])
    )  # A[N, N-1]
    AN2 = zero if endpoints == "natural" else np.array([h[-1]])  # A[N, N-2]

    # Construct the tri-diagonal matrix A
    A = np.diag(np.concatenate((A00, 2 * (h[:-1] + h[1:]), ANN)))
    upper_diag1 = np.diag(np.concatenate((A01, h[1:])), k=1)
    upper_diag2 = np.diag(np.concatenate((A02, np.zeros(n_data - 3))), k=2)
    lower_diag1 = np.diag(np.concatenate((h[:-1], AN1)), k=-1)
    lower_diag2 = np.diag(np.concatenate((np.zeros(n_data - 3), AN2)), k=-2)
    A += upper_diag1 + upper_diag2 + lower_diag1 + lower_diag2

    # Construct RHS vector s
    center = 3 * (p[1:] / h[1:] - p[:-1] / h[:-1])
    s = np.concatenate((zero, center, zero))
    # Compute spline coefficients by solving the system
    coefficients = np.linalg.solve(A, s)

    # Compute the spline coefficients for a given x
    knots = x

    # Include the right endpoint in spline piece C[m-1]
    ind = np.clip(ind, 0, len(knots) - 2)
    h = np.diff(knots)[ind]

    c = coefficients[ind]
    c1 = coefficients[ind + 1]
    a = y[ind]
    a1 = y[ind + 1]
    b = (a1 - a) / h - (2 * c + c1) * h / 3.0
    d = (c1 - c) / (3 * h)

    # Evaluation of the spline.
    return a * t_0 + b * t_1 + c * t_2 + d * t_3


In [9]:
# Now the name of the game is to calculate the 4 spline coefficients in each region, multiply them by the summed_..._diffs
# and finally, sum over all splines
ell = 100
m = 33
a_lm = np.sum(cubic_spline(theta_samples, Ylm_np[ell, m, :], np.arange(1024),
                           summed_0_diffs, summed_1_diffs, summed_2_diffs, summed_3_diffs))

# After that, we just loop over ell and m. And this is actually trivially parallelizable

In [10]:
%%timeit
np.sum(cubic_spline(theta_samples, Ylm_np[ell, m, :], np.arange(1024),
                    summed_0_diffs, summed_1_diffs, summed_2_diffs, summed_3_diffs))

18 ms ± 190 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
